<a href="https://colab.research.google.com/github/avishank-dwivedi/Pytorch-Framework/blob/main/13RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


In [2]:
from google.colab import drive


# Mount Drive
drive.mount('/content/drive')

# Read the file (just adjust the path)
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/pytorch/100_Unique_QA_Dataset.csv')
df.head()

Mounted at /content/drive


,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [5]:
#tokenizer
def tokenizer(text):
  test = text.lower()
  text = text.replace('?','')
  text = text.replace("'" , "")
  return text.split()

In [6]:
tokenizer('Is the theme optimized for mobile devices?')

['Is', 'the', 'theme', 'optimized', 'for', 'mobile', 'devices']

In [7]:
#vocab
vocab = {'<UNK>':0}
def build_vocab(row):

  tokenizer_question = tokenizer(row['question'])
  tokenizer_answer = tokenizer(row['answer'])

  merged_tokens = tokenizer_question + tokenizer_answer

  for token in merged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)





In [8]:
df.apply(build_vocab , axis = 1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [9]:
#convert words to numerical indices

def text_to_indices(text ,vocab):
  indexed_text = []

  for token in tokenizer(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])
  return indexed_text




In [10]:
text_to_indices("Is the theme optimized for mobile devices?",vocab)

[0, 3, 0, 0, 39, 0, 0]

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader

In [12]:
class QADataset(Dataset):
  def __init__(self , df , vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self , index):
    numerical_question = text_to_indices(self.df.iloc[index]['question'] , self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'] , self.vocab)

    return torch.tensor(numerical_question) , torch.tensor(numerical_answer)


In [13]:
dataset = QADataset(df , vocab)

In [14]:
dataloader = DataLoader(dataset , batch_size = 1 , shuffle = True )

In [24]:
for question , answer in dataloader:
  print(question , answer[0])

tensor([[ 42, 141,   2, 142,  39, 143]]) tensor([54])
tensor([[  1,   2,   3,   4,   5, 115]]) tensor([116])
tensor([[ 10,  29, 134, 135]]) tensor([136])
tensor([[1, 2, 3, 4, 5, 8]]) tensor([9])
tensor([[ 10,  98,   3, 106, 244]]) tensor([245])
tensor([[10, 29,  3, 30, 31]]) tensor([32])
tensor([[  1,   2,   3, 185, 186, 187, 188]]) tensor([189])
tensor([[  1,   2,   3, 217,   5,  14, 218, 219]]) tensor([220])
tensor([[ 42, 172,   2,   3,  17, 173, 174]]) tensor([175])
tensor([[ 10,  77,   3, 302,  19, 303]]) tensor([304])
tensor([[ 42, 103,   2,   3,  17]]) tensor([104])
tensor([[  1,   2,   3, 105,   5, 106,  19, 107]]) tensor([108])
tensor([[ 10,   2,  63,  64,   3, 289,   5, 290]]) tensor([291])
tensor([[10, 56,  3, 57,  5, 58]]) tensor([59])
tensor([[ 42, 268, 269,  14, 270, 271, 163, 272]]) tensor([273])
tensor([[ 80,  81, 294,  83,  19,  14, 295]]) tensor([87])
tensor([[ 42, 141,   2,  63,  39,   3, 328, 329]]) tensor([6])
tensor([[42, 43, 44, 45, 46, 47, 48]]) tensor([49])
tens

In [25]:
import torch.nn as nn


In [35]:
class SimpleRNN(nn.Module):
  def __init__(self , vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size , embedding_dim = 50)
    self.rnn = nn.RNN(50, 64, batch_first=True) # Add batch_first=True
    self.fc = nn.Linear(64 , vocab_size)

  def forward(self , question):
    embedded_question = self.embedding(question)
    hidden , final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))



    return output

In [36]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [37]:
learning_rate = 0.001
epochs = 20

In [38]:
model = SimpleRNN(len(vocab))

In [39]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters() , lr = learning_rate)

In [40]:
#training loop

for epoch in range(epochs):
  total_loss = 0

  for question , answer in dataloader:
    optimizer.zero_grad()

    # forward pass
    output = model(question)


    # loss
    loss = criterion(output, answer[0])

    #gradient
    loss.backward()

    #optimizer
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch {epoch+1} , Loss {total_loss:4f}")

Epoch 1 , Loss 527.835549
Epoch 2 , Loss 461.987161
Epoch 3 , Loss 384.646560
Epoch 4 , Loss 318.176036
Epoch 5 , Loss 266.576741
Epoch 6 , Loss 218.027402
Epoch 7 , Loss 173.773589
Epoch 8 , Loss 135.664733
Epoch 9 , Loss 103.139863
Epoch 10 , Loss 79.019380
Epoch 11 , Loss 60.517730
Epoch 12 , Loss 47.298851
Epoch 13 , Loss 37.935087
Epoch 14 , Loss 30.410884
Epoch 15 , Loss 25.053473
Epoch 16 , Loss 20.710624
Epoch 17 , Loss 17.532779
Epoch 18 , Loss 14.859235
Epoch 19 , Loss 12.673813
Epoch 20 , Loss 11.025276


In [41]:
dataset[0][0]

tensor([1, 2, 3, 4, 5, 6])

In [42]:
x = nn.Embedding(324 , embedding_dim = 50)

In [43]:
a = x(dataset[0][0])

In [44]:
a

tensor([[-0.6172,  0.3243, -0.3686, -1.7659,  1.2906,  1.0235, -0.7728,  0.7225,
         -0.1870,  0.1488,  0.0472, -0.9393, -0.7688, -0.8547, -2.1375,  0.2744,
         -0.7245,  0.8787, -0.2636, -0.4640,  0.8666,  0.1857, -0.3694,  0.6802,
          1.4565,  0.7600,  0.3792, -0.4732, -1.7088,  2.0487, -0.7011, -0.1272,
         -1.4255,  0.7914,  1.0877,  2.0183, -0.5746,  1.0130, -0.4145,  0.5740,
          0.1632,  1.5309,  1.5430,  0.6930, -0.1992,  1.7627,  1.3085,  1.1312,
          0.3251, -0.6989],
        [-0.5805, -0.2646, -0.7876,  1.6002, -0.1185,  1.3550, -0.3294, -0.8243,
          0.4590,  2.1426, -0.9060, -0.1879, -0.8611, -1.6536,  1.5162,  0.5254,
          1.3127, -0.8092,  0.5910,  0.1183, -0.0946,  0.6340,  1.1360,  0.1906,
         -1.1326, -1.6813,  1.8757, -0.5279, -1.9669,  0.0083,  1.9780, -0.9417,
         -1.4244,  0.0906, -0.1758, -1.1140,  1.1573,  0.4054, -0.3393, -1.3049,
         -2.1114,  0.6926,  0.8290,  1.3332, -1.8414,  0.8256,  0.1660,  0.1313,


In [45]:
#hideen
y(a)[0]

tensor([[-4.1898e-01, -1.6477e-01,  6.3736e-01,  7.1459e-01,  6.0833e-01,
          3.0208e-01, -3.0570e-01, -1.4144e-02,  2.5777e-01,  2.1172e-01,
         -4.7525e-01,  5.5579e-01, -6.5647e-01, -4.1426e-01,  2.4449e-01,
          2.3664e-01, -1.1630e-01,  4.8952e-01,  6.2567e-01,  3.9194e-03,
         -2.7145e-01, -9.5935e-02, -3.2482e-01, -1.9655e-01,  2.1052e-01,
         -3.6577e-02,  4.4710e-01,  8.3204e-01,  5.8832e-01, -6.3858e-01,
          2.3778e-01, -4.5751e-01, -1.4358e-01, -4.8510e-01,  5.3388e-01,
         -1.5240e-01,  4.6359e-01,  8.0313e-01,  2.9700e-01, -2.7239e-02,
         -4.5080e-01, -3.3380e-01, -2.2769e-02,  3.4166e-01, -1.0316e-01,
          5.9677e-01,  4.9460e-01, -6.3101e-02,  2.0771e-01, -6.6462e-01,
         -5.8169e-01, -3.9007e-01,  2.3440e-01, -1.2622e-01,  7.0548e-01,
         -5.5808e-02, -1.9553e-01,  1.0320e-01,  2.6513e-01, -3.3575e-01,
          1.8525e-01, -4.1205e-01, -2.2905e-01,  3.4061e-01],
        [-1.0484e-01, -2.6232e-01, -3.3180e-01,  5

In [50]:
import torch.nn.functional as F

def predict(model , question ,threshold=0.5):

  numerical_question = text_to_indices(question , vocab)

  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  output = model(question_tensor)

  probs = F.softmax(output, dim =1)

  value,index = torch.max(probs,dim=1)

  if value < threshold:
    print("i don't know")
  print(list(vocab.keys())[index])

In [51]:
predict(model, "What is the largest planet in our solar system?")

Jupiter


In [53]:
list(vocab.keys())[7]

'Paris'